In [ ]:
# Modules for deep learning and other operations
import os
from math import floor, ceil
import numpy as np
import torch
from torchvision import transforms
import pytorch_lightning as pl
# Modules with networks
import sys
sys.path.append('../../')
from noise_models.cnn import PixelCNN
from decoder.net import SDecoder
from LVAE.models.lvae import LadderVAE
from utils.datasets import TrainDatasetUnsupervised, PredictDatasetVAE
# Modules for loading and viewing images
import tifffile
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load the noisy images
# These will be a list of PyTorch tensors with dimensions: [Channels, Height, Width]
top_dir = '../../data/Fluo-C2DL-MSC/low_snr'
files = os.listdir(top_dir)
low_snr = [tifffile.imread(os.path.join(top_dir, file)) for file in files]
low_snr = [torch.from_numpy(image) for image in low_snr]

In [ ]:
# Calculate mean and standard deviation for normalisation
cumulative_sum = 0
cumulative_size = 0
for i in range(len(low_snr)):
    cumulative_sum += low_snr[i].sum()
    cumulative_size += low_snr[i].numel()

low_snr_mean = cumulative_sum / cumulative_size

cumulative_deviation = 0
for i in range(len(low_snr)):
    cumulative_deviation += ((low_snr[i] - low_snr_mean)**2).sum()

low_snr_std = (cumulative_deviation / cumulative_size)**0.5

In [ ]:
# Check
idx = np.random.randint(len(low_snr))
plt.imshow(low_snr[idx][0])
plt.show()

In [ ]:
# Create training, validation and prediction dataloaders
batch_size = 4
crop_size = 256
n_iters = max(low_snr[0].shape[-2], low_snr[0].shape[-1]) // crop_size
transform = transforms.RandomCrop(crop_size)

train_val_set = TrainDatasetUnsupervised(low_snr, n_iters=n_iters, transform=transform)
train_set, val_set = torch.utils.data.random_split(train_val_set, (floor(0.8*len(train_val_set)), ceil(0.2*len(train_val_set))))
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

predict_set = [image[:, :512, :512] for image in low_snr]

n_samples = 10 # The number of times the same image should be evaluated on
predict_set = PredictDatasetVAE(predict_set, n_samples=n_samples)
predict_loader = torch.utils.data.DataLoader(predict_set, batch_size=n_samples, shuffle=False)

In [ ]:
# The number of features the clean signal code will have, both the noise model and the signal decoder will need to know this
s_code_features = 32

# Create the kernel mask for the noise model.
# This mask will allow 15 vertically adjacent pixels into the receptive field.
kernel_mask = torch.zeros((1, 16))
kernel_mask[-1, :] = 1
kernel_mask[:, -1] = 1
kernel_mask[-1, -1] = 0

# Create the noise model.
noise_model = PixelCNN(colour_channels=1,
                       code_features=s_code_features,
                       kernel_mask=kernel_mask,
                       n_filters=32,
                       n_layers=4,
                       n_gaussians=5)

# Create the signal decoder.
s_decoder = SDecoder(colour_channels=1, code_features=s_code_features)

# Create the VAE
# Pass the noise model and signal decoder as an argument
vae = LadderVAE(colour_channels=1,
                data_mean=low_snr_mean,
                data_std=low_snr_std,
                img_shape=(crop_size, crop_size),
                noise_model=noise_model,
                s_decoder=s_decoder,
                n_filters=s_code_features)

In [ ]:
# Where training logs and the trained parameters will be saved
checkpoint_path = '../../checkpoints/Fluo-C2DL-MSC/DVLAE/'

# Create the pytorch lightning trainer
trainer = pl.Trainer(default_root_dir=checkpoint_path,
                     accelerator='gpu',
                     devices=1,
                     max_epochs=500,
                     log_every_n_steps=len(train_set) // batch_size)

In [ ]:
# Train and save model
trainer.fit(vae, train_loader, val_loader)
trainer.save_checkpoint(os.path.join(checkpoint_path, 'final_params.ckpt'))

In [ ]:
# Load the trained model
# Note the new hyperparameters: the noise model is no longer necessary and the loss does not need to be calculated so <mode_pred> is set True. The image shape <img_shape> is the same this time but would need to be changed if we were to evaluate on images of a different size to the training ones.
vae = LadderVAE.load_from_checkpoint(os.path.join(checkpoint_path, 'final_params.ckpt'),
                                     noise_model=None,
                                     s_decoder=s_decoder,
                                     mode_pred=True,
                                     img_shape=(512, 512),
                                     strict=False).eval()

# Evaluate the model on the test set
# The returned "predictions" will be a list with length equal to the number of noisy images.
# Each element of the list is a tensor of shape [n_samples, colours, height, width] and is n_samples many signal estimates for a noisy image
predictions = trainer.predict(vae, predict_loader)

# To get an MMSE, we take the mean of each batch returned by predict()
MMSEs = [samples.mean(dim=0) for samples in predictions]

In [ ]:
# Save results to be used as pseudo ground truth for training HDN36
for i in range(len(MMSEs)):
    tifffile.imwrite('../../data/Fluo-C2DL-MSC/DVLAE_results/' + str(i) + '.tif', MMSEs[i].numpy())

In [ ]:
# Select a random image, its corresponding clean estimates and its ground truth
idx = np.random.randint(len(predictions))

noisy_image = predict_set[idx * n_samples][0]
MMSE = MMSEs[idx][0]
samples = predictions[idx]

# Select a random sample
sample_idx = np.random.randint(n_samples)
sample = samples[sample_idx, 0]

vmin = np.percentile(noisy_image, 1)
vmax = np.percentile(noisy_image, 99.99)

# Display all three
fig, ax = plt.subplots(1, 3, figsize=[15, 15])
ax[0].imshow(noisy_image, cmap='inferno', vmin=vmin, vmax=vmax)
ax[0].axis('off')
ax[0].set_title('Input')

ax[1].imshow(MMSE, cmap='inferno', vmin=vmin, vmax=vmax)
ax[1].axis('off')
ax[1].set_title('MMSE')

ax[2].imshow(sample, cmap='inferno', vmin=vmin, vmax=vmax)
ax[2].axis('off')
ax[2].set_title('Sample')

fig.show()